In [1]:
import pandas as pd
import numpy as np
import datetime as dt

pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
## EXAMPLE
URL = "https://spreadsheets.google.com/tq?tqx=out:html&tq=&key=14yFTnITXiT8c-DVd1InqguhPqpcNFjh9_y_qNButXQQ"
df = pd.read_html(URL)[0]
df.columns = df.iloc[0]
df = df[1:]

df.dropna(axis=1, how='all',inplace=True)
df.dropna(axis=0, how='all',inplace=True)

In [20]:
def read_gs(gs_url):
    df = pd.read_html(gs_url,)[0]
    df.columns = df.iloc[0]
    df = df[1:]
    df.dropna(axis=1, how='all',inplace=True)
    df.dropna(axis=0, thresh=5,inplace=True)
    return df

In [24]:
#url_test = "https://docs.google.com/spreadsheets/d/1Ts6BSGN3XE3bgJaUpeNZAd3foaBRCtiwDKPdcpAGiNA/edit?usp=sharing"
#url_test = 'https://docs.google.com/spreadsheets/d/1Ts6BSGN3XE3bgJaUpeNZAd3foaBRCtiwDKPdcpAGiNA/edit?usp=sharing'
url_test = 'https://docs.google.com/spreadsheets/d/14yFTnITXiT8c-DVd1InqguhPqpcNFjh9_y_qNButXQQ/edit?usp=sharing'

In [25]:
df = read_gs(url_test)

In [26]:
df.head()

,1.0,Date,Country,Type,Asset,Class,Ticker,Action,No.,Price,Commission,Value,Value(Baht),Current(Baht),Current Price,Percent,Annualized,Gain/Loss,Comment
2,3.0000,11/2/2020,us,s,eq,s_eq,MSFT,B,75,202.40,0,15180.00,478170.00,508480.88,215.23,6.34%,122.82%,30310.88,NaN
3,4.0000,11/2/2020,us,s,eq,s_eq,MA,B,35,292.24,0,10228.40,322194.60,373824.68,339.07,16.02%,594.15%,51630.08,NaN
4,5.0000,10/6/2020,th,s,eq,s_eq,M.BK,B,10000,45.58,0,455764.00,455764.00,557500.00,55.75,22.32%,280.81%,101736.00,NaN
5,9.0000,5/27/2020,th,s,eq,s_eq,BBL.BK,B,5000,102.67,770.04,512589.96,512589.96,612500.00,122.50,19.31%,41.15%,99910.04,NaN
6,11.0000,4/27/2020,th,s,eq,s_eq,SABINA.BK,B,20000,16.90,0,338000.00,338000.00,426000.00,21.30,26.04%,47.58%,88000.00,NaN


In [ ]:
## FORMAT DATA
DATE_COL = 'date'
TICKER_COL = 'ticker'
ACTION_COL = 'action'
NUM_COL = 'no.'
PRICE_COL = 'price'

df.columns = [str(x).lower() for x in df.columns]

all_tickers = sorted(list(df[TICKER_COL].unique()))

df[DATE_COL] = pd.to_datetime(df[DATE_COL]).dt.date
begin_date, end_date = df[DATE_COL].min(), df[DATE_COL].max()
date_ls = [begin_date+dt.timedelta(days=k-1) for k in range((end_date - begin_date).days+2)]
print(date_ls[0],date_ls[-1],begin_date, end_date)
df.sort_values(by=DATE_COL,inplace=True)

df[NUM_COL] = df[NUM_COL].astype(float) 
df[PRICE_COL] = df[PRICE_COL].astype(float)

In [ ]:
from pandas_datareader import data as pdr
def get_price_data(ticker, start_date, end_date):
    """

    :param ticker: ticker of the stock in Yahoo
    :type ticker: str (eg. cpall, PTT)
    :param start_date: start date of the data
    :type start_date: str in the format 'YYYY-MM-DD'
    :param end_date: the last date of the data
    :type end_date str in the format 'YYYY-MM-DD'
    :return: pd.DataFrame
    """
    # DOWNLOADING DATA
    ticker_yh = ticker.upper()
    data = pdr.get_data_yahoo(
        ticker_yh,
        start=start_date, end=end_date
    ).reset_index()
    data['yahoo_symbol'] = ticker_yh
    return data

In [ ]:
import datetime as dt
min_date_str = dt.date.strftime(date_ls[0]-dt.timedelta(days=7), '%Y%m%d')
max_date_str = dt.date.strftime(dt.date.today(), '%Y%m%d')
df_price = pd.DataFrame()
for ticker in all_tickers:
    try:
        dfs = get_price_data(ticker,start_date=min_date_str,end_date=max_date_str)
        df_price = df_price.append(dfs)
        print(ticker, df_price.shape)
    except:
        print('ERROR', ticker)

In [ ]:
df_price.to_csv("price_data_test.csv")

In [ ]:
def get_action_sign(a_str):
    if a_str.lower().startswith('b'):
        return 1
    if a_str.lower().startswith('s'):
        return -1
    print(f"Unknown action {a_str}: return zero")
    return 0

def is_local_ticker(ticker):
    return ticker.endswith('.BK')

In [ ]:
print(begin_date, end_date, date_ls[0], date_ls[-1])
import copy
ticker_ls = sorted(list(df[TICKER_COL].unique()))
initial_cash = 1e6
USD_THB = 31.0

cur_pos = {k:0 for k in ticker_ls}
cur_pos['cash'] = initial_cash
pos = {d:copy.copy({k:np.nan for k in ticker_ls}) for d in date_ls}
pos[date_ls[0]]['cash'] = initial_cash
cur_d = date_ls[0]-dt.timedelta(days=1)
for r in zip(df[DATE_COL], df[TICKER_COL], df[ACTION_COL], df[NUM_COL], df[PRICE_COL]):
    #update position of ticker and date
    d, ticker, a_str, num, p = r
    a = get_action_sign(a_str)
    p_local = p*USD_THB if not is_local_ticker(ticker) else p
    #print(d, ticker, a_str, a,num, p, p_local)
    if d > cur_d:
        pos[cur_d] = copy.copy(cur_pos)
        cur_d = d
    pos[d][ticker] = cur_pos[ticker] + a*num
    cur_pos['cash'] =  cur_pos['cash'] - a*num*p_local
    cur_pos[ticker] = pos[d][ticker]
df_pos = pd.DataFrame(pos).T.sort_index().fillna(method='ffill').fillna(0)  

idx = ~df_price['yahoo_symbol'].str.contains('.BK')
df_price['adj_close_local'] = df_price['Adj Close']
df_price.loc[idx,'adj_close_local'] = df_price.loc[idx,'Adj Close']*USD_THB
df_price_tb = df_price.pivot_table(values='adj_close_local', index='Date', columns='yahoo_symbol')
df_price_tb['cash'] = 1.0
diff_set = set(df_pos.columns).difference(df_price_tb.columns)
print(diff_set)
assert diff_set == set()
df_price_tb = df_price_tb[df_pos.columns]

df_pos = df_pos.merge(df_price_tb.fillna(method='ffill'), how='left', left_index=True, right_index=True, suffixes=('','_Price'))
df_pos = df_pos.fillna(method='ffill')

In [ ]:
cols = ticker_ls
p_cols = [f"{x}_Price" for x in cols] 
df_pos['portfolio_value'] = df_pos[cols].multiply(df_pos[p_cols]).sum(axis=1)

In [ ]:
df_value = df_pos[cols].multiply(df_pos[p_cols].rename(columns=dict(zip(p_cols,cols))))
df_value.columns = [f"{x}_Value" for x in cols]

In [ ]:
df_value.tail(10).T.sort_values(by='2020-11-02')

In [ ]:
df_pos['portfolio_value'] = df_pos[cols].multiply(df_pos[p_cols].rename(columns=dict(zip(p_cols,cols)))).sum(axis=1)
df_pos['portfolio_value_with_cash'] = df_pos['portfolio_value'] + df_pos['cash']

In [ ]:
plt.figure(figsize=(20,10))
df_pos['portfolio_value'].plot()
plt.figure(figsize=(20,10))
df_pos['portfolio_value_with_cash'].plot()

In [ ]:
df_pos.tail(10).T